
# <center>Resection problem in 3D</center>

  \----------------------------------------------------------  
Author: Jorge Ventura Gil  
Date:   October, 2023  
License: BSD-3  
Version: 1.0  
  \----------------------------------------------------------  

The first step is to include the libraries and dependencies to work with GA.

In [2]:
from clifford import *
from clifford.g3c import *
from clifford.tools.g3c import *
from math import *
#from pyganja import *

### Functions


- **GetAngles**    
This auxiliary function is in charge of obtaining the angles alpha, beta and gamma knowing the position of the points A, B, C and P. 

- **VGAMethod_2D**  
The function applies the VGA-based method for solving the 2D rejection problem.

- **CGAMethod_2D**  
The function applies the CGA-based method for solving the 2D rejection problem.


In [3]:
def GetAngles(A, B, C, P):
    
    ua = [A[0] - P[0], A[1] - P[1]]
    ub = [B[0] - P[0], B[1] - P[1]]
    uc = [C[0] - P[0], C[1] - P[1]]
    
    theta_ua = atan2(ua[1], ua[0])
    theta_ub = atan2(ub[1], ub[0])
    theta_uc = atan2(uc[1], uc[0])
    
    
    print("Angles ----> A: %f, B: %f, C: %f\n"%(theta_ua*180/pi, theta_ub*180/pi, theta_uc*180/pi))
    
    alpha =  theta_ua - theta_ub
    beta =  theta_ub - theta_uc
    
    # If P is aligned with A and B or B and C, a new assignment of the points is necessary.
    if beta == 0:
        alpha =  theta_ub - theta_ua
        beta =  theta_ua - theta_uc     
        print("The angle beta = 0 ---> The position of A and B will be interchanged.")
        A[:], B[:] = B[:], A[:]
        
    if alpha == 0:
        alpha =  theta_ua - theta_uc
        beta =  theta_uc - theta_ub
        
        print("The angle alpha = 0 ---> The position of C and B will be interchanged.")
        C[:], B[:] = B[:], C[:]
    
    # if we are working in 2 dimensions, gamma is not used.
    gamma = None
    if len(A) == 3:
        gamma = atan2(A[2] - P[2], sqrt(ua[0]**2 + ua[1]**2))
        

    return alpha, beta, gamma

#=========================================================================================================

def VGA_Method_2D(A,B,C,alpha,beta):
    
    print("\n================\nVGA-based Method\n================\n")
    
    v1 = (A[0]-B[0])*e1 + (A[1]-B[1])*e2
    v2 = (C[0]-B[0])*e1 + (C[1]-B[1])*e2
        
    d1 = v1 + (v1/tan(alpha))*e12
    d2 = v2 - (v2/tan(beta))*e12
    d = d2-d1
    
    try:
        p = (d1^d)|d.inv()
        P = [B[0] + p.as_array()[1], B[1] + p.as_array()[2]]
        
    except Exception as err:
        print("Prohibited circle ---> ",err)
        
    return P

#=========================================================================================================
    
def CGA_Method_2D(A,B,C,alpha,beta):

    if len(A) == 2: print("\n================\nCGA-based Method\n================\n")
    
    a = A[0]*e1 + A[1]*e2 
    b = B[0]*e1 + B[1]*e2
    c = C[0]*e1 + C[1]*e2

    A = up(a)
    B = up(b)
    C = up(c)

    AB = sqrt(-2*(A << B))
    BC = sqrt(-2*(C << B))

    MA = AB/(2*tan(alpha))
    MB = BC/(2*tan(beta))

    midAB = (((A-B) ^ e3).dual()).normal()
    midBC = (((B-C) ^ e3).dual()).normal()

    S1 = (((A^B)*einf*(A^B)))
    S2 = (((C^B)*einf*(C^B)))

    O1 = up(down(S1) + (((eo^einf) << midAB))*MA)
    O2 = up(down(S2) + (((eo^einf) << midBC))*MB)

    R1 = (math.sqrt(-2*(O1 << B)))
    R2 = (math.sqrt(-2*(O2 << B)))

    C1 = (O1 - 1/2*(R1**2)*einf).normal()
    C2 = (O2 - 1/2*(R2**2)*einf).normal()

    CC = (C1 ^ C2 ^ e3).dual()

    P = (down(point_pair_to_end_points(CC)[0]))

    if P == b:
        P = (down(point_pair_to_end_points(CC)[1]))

    return P


### Examples

1. Point P outside the triangle formed by A, B and C

In [4]:
A = [-7,-1]
B = [1,5]
C = [15.6,6]
P = [3.12,-18.5]

2. Point P inside the triangle formed by points A, B, and C

In [46]:
A = [-5,4]
B = [3.5,10]
C = [7.6,-1]
P = [3,3.5]

3. Point P aligned with A and C

In [48]:
A = [5,6]
B = [0,1]
C = [3,12]
P = [8,-3]

Choose a set of points and run the following code to check how each method is able to correctly calculate the position of point P

In [5]:
alpha, beta, _ = GetAngles(A,B,C,P)
conv = 180/pi
print("ALPHA: %fº\nBETA: %fº"%(alpha*conv,beta*conv))

P = CGA_Method_2D(A,B,C,alpha,beta)
print("The position of point P is: [%f, %f]"%(P.as_array()[1],P.as_array()[2]))

P = VGA_Method_2D(A,B,C,alpha,beta)
print("The position of point P is: [%f, %f]"%(P[0],P[1]))

Angles ----> A: 120.040182, B: 95.154857, C: 63.006264

ALPHA: 24.885325º
BETA: 32.148593º

CGA-based Method



c:\Users\Jorge\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


The position of point P is: [3.120000,-18.500000]

VGA-based Method

The position of point P is: [3.120000,-18.500000]


## Solving the problem in 3D

### Function
- **CGA_Method_3D**  
This method uses the previously defined function _CGA_Method_2D_ to first make a projection onto the _e<sub>12</sub>_ plane and solve the problem in 2D. With the 2D solution we obtain the remaining coordinate by using the _gamma_ angle.

In [6]:
def CGA_Method_3D(A,B,C,alpha,beta,gamma):
    
    P_v = CGA_Method_2D(A,B,C,alpha,beta)
    P_c = up(P_v)
    
    a = A[0]*e1 + A[1]*e2 + A[2]*e3
    b = B[0]*e1 + B[1]*e2 + B[2]*e3
    c = C[0]*e1 + C[1]*e2 + C[2]*e3
        
    A = up(a)
    B = up(b)  
    C = up(c)

    L = P_c ^ up(P_v + e3) ^ einf
    Na = ((A << L).dual()) ^ einf
    F = (Na.dual() ^ L.dual()).dual()
    g =  up(down(eo << F))

    P = down(g) + e3 * tan(-gamma) * sqrt(-2* (g << A))
    
    # print('\nPv = %s\nL = %s\nNa = %s\nF = %s\ng = %s\n\n'%(P_c,L,Na,F,g))

    return P
    

3D point configuration

In [7]:
A = [1,15,0]
B = [-4,1,3]
C = [3,-8,2.6]
P = [6.81034,-6.63073,14.53887809]

In [8]:
alpha, beta, gamma = GetAngles(A,B,C,P)
conv = 180/pi
print("ALPHA: %f º\nBETA: %f º\nGAMMA: %f º"%(alpha*conv,beta*conv,gamma*conv))

#alpha, beta, gamma = 40*conv**-1, 55*conv**-1, -30*conv**-1
p = CGA_Method_3D(A,B,C,alpha,beta,gamma)
print("The position of point P is: [%f, %f, %f]"%(p.as_array()[1],p.as_array()[2],p.as_array()[3]))

Angles ----> A: 105.035590, B: 144.782755, C: -160.233776

ALPHA: -39.747164 º
BETA: 305.016531 º
GAMMA: -32.988778 º

Pv = (6.81034^e1) - (6.63073^e2) + (44.67366^e4) + (45.67366^e5)
L = (6.81034^e134) + (6.81034^e135) - (6.63073^e234) - (6.63073^e235) + (1.0^e345)
Na = (1.0^e1245)
F = (6.81034^e14) + (6.81034^e15) - (6.63073^e24) - (6.63073^e25) - (1.0^e45)
g = (6.81034^e1) - (6.63073^e2) + (44.67366^e4) + (45.67366^e5)


The position of point P is: [6.810340,-6.630730,14.538878]


44.99999779997937